### Gradient Explorations

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.optimize import minimize, LinearConstraint
from scipy.interpolate import interp1d, LinearNDInterpolator

# Move into the source directory for this notebook to work properly
# Probably want a better way of doing this.
import os
import importlib
os.chdir('../src/')

# Import whatever we need
import disruptivity as dis
import vis.disruptivity_vis as dis_vis
import vis.probability_vis as prob_vis
from vis.plot_helpers import plot_subplot as plot
import data_loader

# Import tokamak Configuartions
from tokamaks.cmod import CONFIG as CMOD
from tokamaks.d3d import CONFIG as D3D

importlib.reload(dis)
importlib.reload(dis_vis)
load_disruptions_mat = data_loader.load_disruptions_mat

In [ ]:
# Loading
cmod_df, cmod_indices = load_disruptions_mat('../data/CMod_disruption_warning_db.mat')

In [ ]:
# Entry dictionary
entry_dict_VDE = {
    'z_error': CMOD["entry_dict"]["z_error"],
    'kappa': CMOD["entry_dict"]["kappa"],
#     'z_error': CMOD["entry_dict"]["z_error"],
}

entry_dict_DENSITY = {
    'q95':CMOD['entry_dict']['q95'],
    'n_e':CMOD['entry_dict']['n_e'],
}

# Hugill
# Compute the murakami parameter
cmod_df['inv_q95'] = 1/cmod_df['q95']
cmod_df['murakami'] = cmod_df['n_e']*0.68/(cmod_df['n_equal_1_mode']/cmod_df['n_equal_1_normalized'])/1e19

entry_dict_H = {
    'murakami':{
        'range':[0,20],
        'axis_name': "$n_e R/B_T \ (10^{19}$m$^{-2}$/T)",
    },
    'inv_q95':{
        'range':[0, 1],
        'axis_name': "$1/q_{95}$",
    },
 }

In [ ]:
from matplotlib.collections import LineCollection

def subplot_draw_trajectory(ax, dataframe, entry_dict, indices, shot):
    
    # Filter the datafram data to get the pulse of interest in the flattop
    disruptive_indices = indices["indices_flattop_disrupt_in_flattop"]
    shotlist_bool = np.isin(dataframe.shot, [shot])
    shot_indices = dataframe[shotlist_bool].index
    overlap = np.array(np.intersect1d(disruptive_indices,shot_indices))
    print(f"Flat Top Starts: {dataframe.time[overlap[0]]} s")
    
    # Get the entry information
    entries = list(entry_dict.keys())
    
    # Create line segments to color individually
    # Based on this example: https://matplotlib.org/stable/gallery/lines_bars_and_markers/multicolored_line.html
    time = dataframe['time'][overlap]
    x = dataframe[entries[0]][overlap]
    y = dataframe[entries[1]][overlap]
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    # Create a continuous norm to map from data points to colors
    norm = plt.Normalize(time.min(), time.max())
    lc = LineCollection(segments, cmap='cool', norm=norm)
    # Set the values used for colormapping
    lc.set_array(time)
    lc.set_linewidth(2)
    
    # Add the collection
    line = ax.add_collection(lc)
    
#     # RHS colorbar for time
#     cbar = fig.colorbar(line, ax=ax,location = 'top')
#     cbar.ax.tick_params()
#     cbar.set_label(label="Pulse Time", size="large")
    
#     # Plot the trajectory
#     ax.plot(dataframe[entries[0]][overlap], dataframe[entries[1]][overlap], c=c)
    
    # Re-apply axis labels and limits if needed.
    ax.set_xlim(entry_dict[entries[0]]['range'])
    ax.set_ylim(entry_dict[entries[1]]['range'])
    ax.set_xlabel(entry_dict[entries[0]]['axis_name'])
    ax.set_ylabel(entry_dict[entries[1]]['axis_name'])
    
def subplot_grad2d(
    ax,
    disruptivity: np.ndarray,
    error: np.ndarray,
    bins: np.ndarray,
    entry_dict: dict,
):
    """Creates a subplot of a 2D disruptivity map.

    Args:
        ax (subplot_type): The matplotlib axes to plot on.
        disruptivity (np.ndarray): The disruptivity histogram of size (ny_bins-1, nx_bins-1).
        error (np.ndarray): The error histogram of size (ny_bins-1, nx_bins-1).
        bins (list): List of the bin edges from the histogram.
        entry_dict (list): List of the entries being plotted.
    """

    # Asserts
    assert len(entry_dict) == 2, "Too many entry_dict dimensions."

    # Parse the dict
    extent = []
    axis_name_list = []
    for key in entry_dict:
        entry = entry_dict[key]

        # Follow the order of the dictionary to find x and y
        # Make sure the range is there
        assert (
            "range" in entry
        ), f"Entry {key} of entry_dict missing range field."
        extent.extend(entry["range"])

        # And the axis names
        assert (
            "axis_name" in entry
        ), f"Entry {key} of entry_dict missing axis_name field."
        axis_name_list.append(entry["axis_name"])

    # The Normal Heatmap
    cax = ax.imshow(
        disruptivity.T,
        cmap="viridis",
        origin="lower",
        aspect="auto",
        extent=extent,
    )

    # Axis Titles
    ax.set_xlabel(axis_name_list[0])
    ax.set_ylabel(axis_name_list[1])

    # Colorbar
    cbar = plt.colorbar(cax)
    cbar.ax.tick_params(labelsize="large")
    cbar.set_label(label="Disruptivity Derivative", size="large")


In [ ]:
# Computing the disruptivity and plotting the regular figure
shotlist=None
entry_dict = entry_dict_2D
indices_n_disrupt, indices_n_total = dis.get_indices_disruptivity(CMOD, cmod_df, cmod_indices, shotlist=shotlist)
args = dis.compute_disruptivity_likelihood(cmod_df, entry_dict, indices_n_total, nbins=35, tau=50, window=25)

In [ ]:
fig,ax = plot('cmod_q95_ne_disruptivity_kaloyannis.png', dis_vis.subplot_disruptivity2d, args)
# Get a pulse's flat top data
# shot = 1140226013 #AT PULSE
shot = 1160930030 # BOB PULSE
# shot = 1120105021 #VDE
subplot_draw_trajectory(ax,cmod_df,entry_dict,cmod_indices,shot)

In [ ]:
'''
Iterative averaging procedure
Hold real data, no data, all disrupted as fixed values
Iteratively loop:
    Conv2D with averaging kernel of size n.
    Replace the fixed values, 
    Check for convergence with last iteration via relative_tol (bin wise).
    Check for max iterations
TODO: CLEAN THIS UP, MAKE A NEW FILE FOR ALL THINGS BOUNDARY AVOIDANCE
'''

from scipy.signal import convolve2d
from scipy.ndimage import gaussian_filter

# Setup
# d_max_mult is the multiplier assigned to
# no_data, disrupted data and out of bounds data
n_dims = len(args[2])
d_array = args[0]
d_max_mult = 1
max_iter=100
rel_tol = 1e-6
n = 11

# Create the fixed points array
fixed_points = np.copy(d_array)
max_fill = d_array[d_array>0].max()*d_max_mult
fixed_points[d_array==-1]= max_fill
fixed_points[d_array==-3]= max_fill

# Create the iter_array
iter_array = np.zeros(fixed_points.shape)
iter_array[d_array!=-2] = fixed_points[d_array!=-2]

# Create the convolutional filter
ave_filter = np.ones([n]*n_dims)/(n**n_dims)

# Iterate
for i in range(max_iter):
    # Step 1: Convolve
    new_iter_array = convolve2d(iter_array,ave_filter,
                                mode='same', 
                                fillvalue=max_fill)
    
    # Step 2: Replace the fixed points
    new_iter_array[d_array!=-2] = fixed_points[d_array!=-2]
    
    # Step 3: Relative Tolerance Check
    # Check for division by 0s in early cycles
    if (new_iter_array!=0).all():
        residuals = abs(new_iter_array-iter_array)/new_iter_array
        if (residuals<=rel_tol).all():
            print(f'Data filling converged after {i} iterations.')
            break
    
    # Save the new iteration
    iter_array = new_iter_array
    
# Run one last smoothing operation on the data with a small Gaussian Filter
iter_array = gaussian_filter(iter_array, sigma=0.5, truncate=3)

# Prep the interpolator
bin_centers = (np.array(args[2])[:,1:]+np.array(args[2])[:,:-1])/2
xx = np.meshgrid(*bin_centers)
interper = scipy.interpolate.RegularGridInterpolator(bin_centers, iter_array,
                                                     method='linear',
                                                     bounds_error=False,
                                                     fill_value=max_fill)

In [ ]:
# Visualize the filling
args2 = list(args)
args2[0]=iter_array
fig,ax = plot('iter_fill.png', dis_vis.subplot_disruptivity2d, args2)

# Get a pulse's flat top data
# shot = 1140226013 # AT Pulse
# shot = 1120105021 #VDE
subplot_draw_trajectory(ax,cmod_df,entry_dict,cmod_indices,shot)
# ax.plot([0,20], [0.5, 0.5], '--', c='red')
# ax.plot([0,20], [0.0, 0.5], '--', c='red')

In [ ]:
# First, let's bring over some bicubic interpolation stuff.
def cubic(dx,f):
    """Computes the cubic interpolation given the lookup vector f.
    
    Inputs: f (np.ndarray (1x4)) the lookup vector.
            dx (double) the dx value in range of [0,1].
            
    Returns: ans (double), the interpolation value.
    """
    mat = [[0, 2, 0, 0],
       [-1, 0, 1, 0],
       [2, -5, 4, -1],
       [-1, 3, -3, 1]]
    mat = np.array(mat)/2.0
    dx_vec = np.array([[1, dx, dx**2, dx**3]])
    
    return (dx_vec@mat@f)[0]

def cubic_deriv(dx, f):
    """Computes the cubic derivative given the lookup vector f.

    Inputs: f (np.ndarray (1x4)) the lookup vector.
            dx (double) the dx value in range of [0,1].

    Returns: ans (double), the interpolation value.
    """
    mat = [[0, 2, 0, 0],
           [-1, 0, 1, 0],
           [2, -5, 4, -1],
           [-1, 3, -3, 1]]
    mat = np.array(mat)/2.0
    dx_vec = np.array([[0, 1, 2*dx, 3*dx**2]])
    
    return (dx_vec@mat@f)[0]

def bicubic(dx, dy, f):
    """Computes the bicubic interpolation given the lookup matrix f.
    
    Inputs: f (np.ndarray (4x4)) the lookup matrix.
            dx (double) the dx value in range of [0,1].
            dy (double) the dy value in range of [0,1].
            
    Returns: ans (double), the interpolation value.
    """
    
    # Complete 4 1D cubics over dx.
    f_y = np.array([cubic(dx, f[i]) for i in range(4)])
    
    # Compute the 1D cubic over dy.
    return cubic(dy,f_y)

def bicubic_grad(dx, dy, f):
    """Computes the bicubic interpolation gradient given the lookup matrix f.
    
    Inputs: f (np.ndarray (4x4)) the lookup matrix.
            dx (double) the dx value in range of [0,1].
            dy (double) the dy value in range of [0,1].
            
    Returns: ans (double), the gradient value.
    """
    
    # Complete 4 1D cubics over dx.
    f_y = np.array([cubic(dx, f[i]) for i in range(4)])

    # Complete 4 1D cubics over dx derivative.
    f_y_deriv = np.array([cubic_deriv(dx, f[i]) for i in range(4)])

    # Compute the 1D cubics over dy and dy derivative.
    # Returns [d/dx, d/dy]
    return [cubic(dy, f_y_deriv), cubic_deriv(dy, f_y)]
    
def get_dx_dy_f(x_grid, y_grid, f_padded, x, y):
    """Get the dx, dy, and f for an interpolation matrix f.
    NEEDS TESTING BUT SHOULD BE OK.
    
    Inputs: stuff
    
    Returns: stuff
    """
    
    # Find the index to the right of the pt
    # Notice that f_padded is padded by 0s
    # on either side, meaning that what would
    # usually be idx_2 is instead idx_0 or the
    # start index of the search
    idx_0 = np.searchsorted(x_grid, x)
    idy_0 = np.searchsorted(y_grid, y)

    # dx, dy
    x_res = x_grid[1]-x_grid[0]
    dx = (x - (x_grid[idx_0]-x_res))/x_res
    y_res = y_grid[1]-y_grid[0]
    dy = (y - (y_grid[idy_0]-y_res))/y_res
    
    # Array
    f = f_padded[idy_0:idy_0+4,idx_0:idx_0+4]
    
    return dx, dy, f

In [ ]:
f_padded = np.pad(args2[0], pad_width=2, mode='edge')


In [ ]:
args = args2

# Jacobian
bins = args[2]
dx = bins[0][1] - bins[0][0]
dy = bins[1][1] - bins[1][0]
bin_centers = [bins[0][:-1]+dx,bins[1][:-1]+dy]

# Need to convert from u,v space to x,y space.
grad_x = scipy.ndimage.sobel(args[0], axis=0)*dx
grad_y = scipy.ndimage.sobel(args[0], axis=1)*dy
norm = np.sqrt(grad_x**2 + grad_y**2)

# Gaussian Filter
def g(x,sig):
    return 1 / (2 * np.pi * np.sqrt(sig)) * np.exp( -(x / sig)**2 / 2 )
def g_x(x,sig):
    return - x / sig**2 *g(x,sig)
sig = 2
x = np.linspace(-5, 5, 5)
g_filter = g(x,sig)
g_x_filter = g_x(x,sig)
# plt.scatter(x, g_filter)
# plt.scatter(x, g_x_filter)

# Instead of doing 2 convolve 1Ds, for now construct a 2D kernel
kernel_x = np.outer(g_filter, g_x_filter)
grad_x = convolve2d(args[0],kernel_x.T,
                    mode='same', 
                    fillvalue=max_fill) * dx
grad_y = convolve2d(args[0],kernel_x,
                    mode='same', 
                    fillvalue=max_fill) * dy



fig,ax = plt.subplots(1,3, figsize = (15,4) , constrained_layout=True)

subplot_grad2d(ax[0], grad_x, *args[1:])
subplot_grad2d(ax[1], grad_y, *args[1:])
subplot_grad2d(ax[2], norm, *args[1:])

ax[0].set_title("Grad X")
ax[1].set_title("Grad Y")
ax[2].set_title("Grad Norm")

In [ ]:
# Visualize the filling
args2 = list(args)
args2[0]=iter_array
fig,ax = plot('quiver_fill.png', dis_vis.subplot_disruptivity2d, args2)

# Get a pulse's flat top data
# shot = 1140226013 # AT Pulse
# shot = 1120105021 #VDE
subplot_draw_trajectory(ax,cmod_df,entry_dict,cmod_indices,shot)
# ax.plot([0,20], [0.5, 0.5], '--', c='red')
# ax.plot([0,20], [0.0, 0.5], '--', c='red')

#Quiver
mesh_x, mesh_y = np.meshgrid(*bin_centers)
ax.quiver(mesh_x, mesh_y, grad_x.T, grad_y.T, angles='xy')

In [ ]:
# Ok so now lets look at probability of disruption for a pulse
# shot = 1140226013 # AT Pulse
# shot = 1120105021 #VDE
data = np.array(cmod_df[entry_dict.keys()][cmod_df.shot == shot])
y = dis.p_data(interper(data),1,True)

# Assume the last data point is the disruption time
disr_index = cmod_df.time_until_disrupt.index[cmod_df.shot == shot][-1]

# Make the plot
fig,ax = plt.subplots(1,4, figsize = (4*5,4) , constrained_layout=True)

ax[0].plot(cmod_df.time[cmod_df.shot == shot],y, label=str(shot))
ax[0].plot([cmod_df.time[disr_index],cmod_df.time[disr_index]],[0,1], '--', color='orange', label="Disruption Time")

ax[0].set_xlabel("Time (s)")
ax[0].set_ylabel("Disruption Probability")
ax[0].set_ylim(-0,1.)
ax[0].grid()
ax[0].legend()
ax[0].set_title("Disruption Probability")

# Mean time to disruption as a function of time?
ax[1].plot(cmod_df.time[cmod_df.shot == shot],cmod_df.time[cmod_df.shot == shot]+1/interper(data), label=str(shot)+" $1/d+t_{pulse}$")
ax[1].plot(cmod_df.time[cmod_df.shot == shot], 1/interper(data),linestyle='dashdot', label=str(shot)+" $1/d$" , color='red')
ax[1].plot([cmod_df.time[disr_index],cmod_df.time[disr_index]],[0,100], '--', color='orange', label="Disruption Time")
ax[1].plot([0,1.5],[cmod_df.time[disr_index],cmod_df.time[disr_index]], '--', color='orange')

ax[1].set_xlabel("Time (s)")
ax[1].set_ylabel("Predicted Disruption Time (s)")
ax[1].set_ylim(0,15)
ax[1].set_xlim(0,1.5)
ax[1].grid()
ax[1].legend()
ax[1].set_title("Time To Disruption")

# Gradient plot

pad_grad_x = np.pad(grad_x,1)
pad_grad_y = np.pad(grad_y,1)
indx = np.searchsorted(bins[0], data[:,0])
indy = np.searchsorted(bins[1], data[:,1])
norm_data = np.sqrt(pad_grad_x[indx,indy]**2 + pad_grad_y[indx,indy]**2)


ax[2].plot(cmod_df.time[cmod_df.shot == shot], pad_grad_x[indx,indy])
ax[2].plot([cmod_df.time[disr_index],cmod_df.time[disr_index]],[0,1], '--', color='orange', label="Disruption Time")
ax[2].set_xlabel("Time (s)")
ax[2].set_ylabel("Predicted Disruption Time (s)")
# ax[2].set_ylim(0,15)
ax[2].set_xlim(0,1.5)
ax[2].grid()
ax[2].legend()
ax[2].set_title("Grad X")

ax[3].plot(cmod_df.time[cmod_df.shot == shot], pad_grad_y[indx,indy])
ax[3].plot([cmod_df.time[disr_index],cmod_df.time[disr_index]],[0,1], '--', color='orange', label="Disruption Time")
ax[3].set_xlabel("Time (s)")
ax[3].set_ylabel("Predicted Disruption Time (s)")
# ax[2].set_ylim(0,15)
ax[3].set_xlim(0,1.5)
ax[3].grid()
ax[3].legend()
ax[3].set_title("Grad Y")

fig.savefig("full_sweep.png", dpi=400)

In [ ]:
norm_data